In [2]:
import pandas as pd
import numpy as np
import warnings
#import base64
from umls_utils import get_paths, CUIbase64

import os
from collections import Counter

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [1]:
!jupyter nbconvert --to script --no-prompt MP_n10s-Copy1.ipynb
!sed -i '' '/.head(/d' MP_n10s-Copy1.py
!sed -i '' '/^#/d' MP_n10s-Copy1.py
!sed -i '' '/get_ipython()/d' MP_n10s-Copy1.py

[NbConvertApp] Converting notebook MP_n10s-Copy1.ipynb to script
[NbConvertApp] Writing 14465 bytes to MP_n10s-Copy1.py


In [3]:
# Path to config file
config_path = '/Users/stearb/Dropbox/CHOP/R03/code/neo4j_build_CFDIKG/build_scripts/'

# Get paths from config file
data_dir,helper_data_dir,output_dir,LOCAL_CPU, umls_dir, umls_out_dir = get_paths(config_path)

if not  os.path.isdir(output_dir+'MPO'):
    os.mkdir(output_dir+'MPO')
    print('Creating MPO directory...')

In [4]:
# check the CSV for duplicates instead of the graph
# check which nbs depend on output data from other nbs
#check that no output files have the same name.

In [5]:
############ Helper files ###########

##### orthologs DONE
# (4) umls_genes  = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-genes-concepts-codes.csv')

##### MP DONE
# (3) /Users/stearb/Desktop/R03_local/data/ontologies/MP_term_name_overlap.csv  terms from MP that already exist in UMLS

#### GTEX
# (1) samp_annos = pd.read_csv('gtex_sample_annotations.txt',sep='\t') # sample annotations ub-tissue mappings,,, #!curl --insecure https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt > gtex_sample_annotations.txt
# (2) we need the original file in gtex , SAME AS IN HGNC ANNOS hgnc_master = pd.read_csv('hgnc_master.txt',sep='\t') # maps gene names to hgnc ids ,,,!curl ftp://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/non_alt_loci_set.txt > hgnc_master.txt
# SAME AS IN ORTHOLOGS: umls_genes  = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-genes-concepts-codes.csv'),,, hgnc ids mapped to their CUIs
# (6) umls_uberon  = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-uberon-code-concept.txt', dtype = dtype_dic), UB codes to their CUIs
# (7) chrom_SUIs, SUIs corresponding to the chromosome Terms

#### IMPC/MGI DONE 
# None

#### HPO-MP DONE
# (5) umls_hpo = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-hpo-code-concepts.csv')

### HGNC Annos DONE
# SAME AS IN GTEX/ORTHOLOGS: umls_genes  = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-genes-concepts-codes.csv')
# same as in gtex: hgnc_master = pd.read_csv('/Users/stearb/Dropbox/CHOP/R03/code/dbSNP/hgnc_master.txt',sep='\t')



###############################
####### DATA files ############

##### orthologs
# hgnc_bulk = pd.read_csv('/Users/stearb/downloads/human_mouse_hcop_fifteen_column.txt',sep='\t')


##### MP
# /Users/stearb/Desktop/R03_local/data/ontologies/MP_n10s_directional.csv  load in n10s graph dump

##### GTEX
# eqtl files '/Users/stearb/desktop/R03_local/data/gtex/GTEx_Analysis_v8_eQTL/egenes/'
# gene_median_tpm = '/Users/stearb/desktop/R03_local/data/gtex/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct'


##### IMPC/MGI
# '/Users/stearb/dropbox/CHOP/R03/code/genotype_phenotype/geno2pheno_MGI_table5.csv'
# stats_file = '/Users/stearb/desktop/R03_local/impc_ftp/statistical-results-ALL.csv'
# gno_phno = '/Users/stearb/desktop/R03_local/impc_ftp/genotype-phenotype-assertions-ALL.csv'
# mgi_master_list = pd.read_csv('mgi_master_geno2pheno.csv') 
## MGI: table_2 = 'http://www.informatics.jax.org/downloads/reports/HMD_HumanPhenotype.rpt'
## MGI: table_5 = 'http://www.informatics.jax.org/downloads/reports/MGI_PhenoGenoMP.rpt'
## MGI: table_9 = 'http://www.informatics.jax.org/downloads/reports/MGI_GenePheno.rpt'
## MGI: table10 = 'http://www.informatics.jax.org/downloads/reports/MGI_Geno_DiseaseDO.rpt'


#### HPO-MP mappings
# (from IMPC_ftp) hpo2mp = pd.read_csv('/Users/stearb/dropbox/CHOP/R03/code/phenotype_mapping/tiffs_mappings_ravel.csv',index_col=0)
# actual data: df=pd.read_excel('/Users/stearb/desktop/R03_local/data/tiffany_mappings/KF_HPO_OBO2OBOMappings_Aggregated_20DEC2020.xlsx',                                                            sheet_name='KF_OMOP2OBMappings_20Dec2020')

#### HGNC_annos
# ucsc = pd.read_csv('/Users/stearb/Desktop/R03_local/data/hgnc_annotations/hgnc_annotations.txt',sep='\t')



Definition of Ontology
Purl


# This notebook is for importing/parsing/formatting a Neo4j database dump of the Mammalian Phenotype Ontology (MPO). The MPO graph was created using the the Neosemantics Plugin for Neo4j.

We are parsing/formatting the data so it can be uploaded into the UMLS knowledge graph.

##  Cypher commands to produce MPO graph and export it to Desktop:

CALL n10s.onto.import.fetch('file:///Users/stearb/Desktop/R03_local/data/ontologies/mp.owl','RDF/XML');

WITH "match(n)-[r]->(m) return n.name as start_code,n.label as start_name,type(r) as rel,m.name as end_code, m.label as end_name"  as query
CALL apoc.export.csv.query(query, "MP_n10s.csv", {})
YIELD done RETURN done;

### Load in just the single SCO relationship by specifying direction in the cypher query, before we were not specifying direction and we were getting relationships and their reciprocals 

# Need to Complete this with the external references...

In [6]:
if LOCAL_CPU:
    mp = pd.read_csv('/Users/stearb/Desktop/R03_local/data/ontologies/MP_n10s_directional.csv')
else:
    mp = pd.read_csv(data_dir+'MP_n10s_directional.csv')

    
print(mp.shape)
mp.sample(5)

(79560, 5)


,start_code,start_name,rel,end_code,end_name
74476,UBERON_0006241,future spinal cord,SCO,UBERON_0006598,presumptive structure
26855,GO_0051345,positive regulation of hydrolase activity,SCO,GO_0051336,regulation of hydrolase activity
27638,GO_0035545,determination of left/right asymmetry in nervo...,SCO,GO_0007368,determination of left/right symmetry
33515,GO_1905333,regulation of gastric motility,SCO,GO_0044058,regulation of digestive system process
12970,GO_0099054,presynapse assembly,SCO_RESTRICTION,GO_0007416,synapse assembly


In [7]:
print(mp.isna().sum())
mp[mp['start_name'].isna()].head(3)

start_code     0
start_name    66
rel            0
end_code       0
end_name      14
dtype: int64


,start_code,start_name,rel,end_code,end_name
700,UBERON_0008817,NaN,SCO,UBERON_0000925,endoderm
2157,UBERON_0000086,NaN,SCO_RESTRICTION,UBERON_0000992,ovary
8842,CvDC_Terms,NaN,SPO,SubsetProperty,subset_property


In [8]:
#Why are these here?
mp[mp['start_code'] ==  'MGI:97753']

,start_code,start_name,rel,end_code,end_name
10419,MGI:97753,Ppy (mouse),SCO,SO_0001217,protein_coding_gene
10762,MGI:97753,Ppy (mouse),SCO_RESTRICTION,NCBITaxon_10090,Mus musculus


In [9]:
# See what the most common Ontologies (SAB) are in MPO
all_codes = pd.concat([mp['start_code'],mp['end_code']])

Counter([i.split('_')[0] for i in np.unique(all_codes)]).most_common(10)

[('MP', 13205),
 ('GO', 9883),
 ('UBERON', 8830),
 ('CHEBI', 2130),
 ('CL', 1410),
 ('PATO', 1036),
 ('PR', 961),
 ('NBO', 400),
 ('MPATH', 209),
 ('RO', 203)]

## We need to load the CUI-CODE mappings in for the ontologies that are already in  UMLS because we dont want to recreate the CUIs if theyre  already there. 


### MUST add these to the use_config/helper_data/ file if we want to include them.

In [10]:
# THESE ARE NOT CURRENTLY BEING ADDED, WE ARE DOING MP ONLY.

# MUST GET THESE FROM CSVs (this way we dont need to build the graph in order to build the graph!) 
# and preferably on the fly 
GO = pd.read_csv('/Users/stearb/Desktop/R03_local/data/ontologies/GO_CUI_CODEs.csv')
UB = pd.read_csv('/Users/stearb/Desktop/R03_local/data/ontologies/UBERON_CUI_CODEs.csv')
CL = pd.read_csv('/Users/stearb/Desktop/R03_local/data/ontologies/CL_CUI_CODEs.csv')

In [11]:
# See what the most common relationships are in MPO
Counter(mp['rel'])

Counter({'SCO': 56020,
         'SCO_RESTRICTION': 23064,
         'SPO': 339,
         'DOMAIN': 64,
         'RANGE': 59,
         'EQC_RESTRICTION': 14})

In [10]:
# Define start and end SABs (just take the first string split by '_', so well get MP, GO, etc.)
mp['start_SAB']  = [i[0] for i in mp['start_code'].str.split('_')]
mp['end_SAB']  = [i[0] for i in mp['end_code'].str.split('_')]

In [11]:
# Format UBERON and CL Codes so they match UMLS UBERON and CL Code format (just the number)

#mp['start_code'] = [i.split('_')[1] if i.startswith('UBERON') else i for i in mp['start_code']]
#mp['end_code'] = [i.split('_')[1] if i.startswith('UBERON') else i for i in mp['end_code']]

#mp['start_code'] = [i.split('_')[1] if i.startswith('CL_') else i for i in mp['start_code']]
#mp['end_code'] = [i.split('_')[1] if i.startswith('CL_') else i for i in mp['end_code']]


# Replace '_' with ':'
#mp['start_code'] = [i.replace('_',':') if i.startswith('GO_') else i for i in mp['start_code']]
#mp['end_code'] = [i.replace('_',':') if i.startswith('GO_') else i for i in mp['end_code']]

# Replace _ with :
mp['start_code'] = [i.replace('_',':') if i.startswith('MP_') else i for i in mp['start_code']]
mp['end_code'] = [i.replace('_',':') if i.startswith('MP_') else i for i in mp['end_code']]

In [12]:
# Filter out everything but the rows that contain MP/UBERON/CL/GO

#mp_import = mp.loc[((mp['start_SAB'] == 'MP') & (mp['end_SAB'] == 'MP')) | \
#                      ((mp['start_SAB'] == 'GO') & (mp['end_SAB'] == 'GO')) | \
#                      ((mp['start_SAB'] == 'UBERON') & (mp['end_SAB'] == 'UBERON')) | \
#                      ((mp['start_SAB'] == 'CL') & (mp['end_SAB'] == 'CL')) 
#                    ]


# Just get MP terms for not
mp_only = mp.loc[((mp['start_SAB'] == 'MP') & (mp['end_SAB'] == 'MP'))]

print(mp_only.isna().sum())

mp_only.dropna(inplace=True) 

mp_only.sample(5)

start_code    0
start_name    2
rel           0
end_code      0
end_name      0
start_SAB     0
end_SAB       0
dtype: int64


,start_code,start_name,rel,end_code,end_name,start_SAB,end_SAB
42316,MP:0003254,bile duct inflammation,SCO,MP:0003252,abnormal bile duct physiology,MP,MP
46336,MP:0012278,abnormal cardiogenic mesoderm morphology,SCO,MP:0014141,abnormal mesoderm morphology,MP,MP
53386,MP:0004286,abnormal internal auditory canal morphology,SCO,MP:0030309,abnormal posterior cranial fossa morphology,MP,MP
40528,MP:0030316,enlarged neurocranium,SCO,MP:0000074,abnormal neurocranium morphology,MP,MP
45721,MP:0008396,abnormal osteoclast differentiation,SCO,MP:0002113,abnormal skeleton development,MP,MP


In [13]:
# Why is this like this?  No xref in MP 
mp.loc[((mp['start_SAB'] == 'UBERON') & (mp['end_SAB'] == 'MP'))]

,start_code,start_name,rel,end_code,end_name,start_SAB,end_SAB


In [14]:
mp_only[mp_only['end_code'] == 'MP:0005385'] # same term mapped to 2 different MP codes

,start_code,start_name,rel,end_code,end_name,start_SAB,end_SAB
48400,MP:0001544,abnormal cardiovascular system physiology,SCO,MP:0005385,cardiovascular system phenotype,MP,MP
48401,MP:0002127,abnormal cardiovascular system morphology,SCO,MP:0005385,cardiovascular system phenotype,MP,MP


In [15]:
# No codes are mapped to themselves which makes sense.
#for n,row in mp_only.iterrows():
#    if row['start_code'] == row['end_code']:
#        print(row['start_code'])

In [16]:
# Drop these 2 cols, Dont need them anymore if were just doing MP
mp_only.drop(['start_SAB','end_SAB'],axis=1,inplace=True)

mp_only.reset_index(drop=True,inplace=True)

Create CodeIDs

In [17]:

#mp['start_CodeID'] = ['MP ' + i for i in mp['start_code']]
#mp['end_CodeID'] = ['MP ' + i for i in mp['end_code']]

mp_only['start_CodeID']  = [i.split(':')[0] + ' '+i for i in mp_only['start_code']]
mp_only['end_CodeID'] = [i.split(':')[0] + ' '+i for i in mp_only['end_code']]

Create and Save CUIs

In [18]:

mp_only['start_CUI'] = [i for i in CUIbase64(mp_only['start_CodeID'])]
mp_only['end_CUI'] = [i for i in CUIbase64(mp_only['end_CodeID'])]

assert len(np.unique(mp_only['start_CUI'])) == mp_only.nunique()['start_CUI']
assert len(np.unique(mp_only['end_CUI'])) == mp_only.nunique()['end_CUI']

CUIs = pd.DataFrame(pd.concat([mp_only['start_CUI'],mp_only['end_CUI']]),columns=['CUI:ID'])

#assert CUIs.duplicated().sum() == 0

# We already checked for collisions, there are duplicates because we are adding the start column and end column 
# together we need to add the end column so we don't miss any of the leaves on the MP tree (because the 
# leaves will only be in the end column)
CUIs.drop_duplicates(inplace=True)


CUIs.to_pickle(output_dir+'MPO/CUIs_mp_ont.pickle')

Create and Save CUI-CUIs

In [19]:
CUI_CUIs = mp_only[['start_CUI','end_CUI','rel']].rename(
                        columns={'start_CUI':':START_ID','end_CUI':':END_ID','rel':':TYPE'})

# change from SCO to isa to match what jonathan did
CUI_CUIs[':TYPE'] = 'isa' 

# Add SAB column
CUI_CUIs['SAB'] = 'MP'

# Create inverse CUI-CUIs 

# Add inverse relationships
CUI_CUIs_inverse = CUI_CUIs.rename(columns={':START_ID':':END_ID',':END_ID':':START_ID'})

CUI_CUIs_inverse[':TYPE'] = 'inverse_isa'

# Combine them.
CUI_CUIs_all = pd.concat([CUI_CUIs,CUI_CUIs_inverse])

CUI_CUIs_all.drop_duplicates(inplace=True)


CUI_CUIs_all.to_pickle(output_dir+'MPO/CUI_CUIs_mp_ont.pickle')

Save CUI-CODEs

In [20]:

CUI_CODEs_array = np.concatenate([mp_only[['start_CUI','start_CodeID']].values,
                                  mp_only[['end_CUI','end_CodeID']].values])

CUI_CODEs = pd.DataFrame(CUI_CODEs_array,columns=[':START_ID',':END_ID'])

CUI_CODEs.drop_duplicates(inplace=True)

CUI_CODEs.to_pickle(output_dir+'MPO/CUI_CODEs_mp_ont.pickle')

Save CODEs

In [21]:

CODEs_array = np.concatenate([mp_only[['start_code','start_CodeID']].values,
                              mp_only[['end_code','end_CodeID']].values])

CODEs = pd.DataFrame(CODEs_array,columns=['CODE','CodeID:ID'])

CODEs['SAB'] = 'MP'

# Reorder
CODEs = CODEs[['CodeID:ID','SAB','CODE']]

CODEs.drop_duplicates(inplace=True)

CODEs.to_pickle(output_dir+'MPO/CODEs_mp_ont.pickle')

Create and Save SUIs


MUST GET THE OVERLAPPING TERMS FROM CSVs (this way we dont need to build the graph in order to build the graph!) and preferably on the fly 

In [22]:
# Read in UMLS SUI CSV
UMLS_SUIs = pd.read_pickle(umls_dir+'SUIs.pickle')
UMLS_SUIs.head(3)


mp_only['start_SUI'] = CUIbase64(mp_only['start_name'])

mp_only['end_SUI'] = CUIbase64(mp_only['end_name'])


SUIs_array = np.concatenate([mp_only[['start_SUI','start_name']].values,mp_only[['end_SUI','end_name']].values])

SUIs = pd.DataFrame(SUIs_array,columns=['SUI:ID','name'])

SUIs.drop_duplicates(inplace=True)

# OLD WAY, when we used cypher to query/save dups
# Only save the Terms that are NOT already in UMLS
#SUIs = SUIs[~SUIs['name'].isin(term_overlap['Term'])] 
### Need to check which Terms are already in UMLS. Load them in first then find duplicates.
#Cypher query to find nodes with duplicate Term names:  
#    -  WITH "match (t:Term)--(c:Code) where not c.SAB = 'MP' with t,c match (x:Term)--(c2:Code) where c2.SAB = 'MP' with t,x,c,c2 where t.name = x.name and t.SUI <> x.SUI  return t.SUI as SUI, t.name as Term, c.SAB as SAB ,x.SUI as SUI_2, x.name as Term_2, c2.SAB as SAB_2" AS query CALL apoc.export.csv.query(query, '/Users/stearb/Desktop/R03_local/data/MP_term_name_overlap.csv', {}) YIELD done RETURN done;


SUIs = SUIs[~SUIs['name'].isin(UMLS_SUIs['name'])] # new way, where we query umls csvs directly

SUIs.to_pickle(output_dir+'MPO/SUIs_mp_ont.pickle')

Save CODE-SUIs (Terms)
Merge in the existing SUIs (for the Terms that were already in UMLS)

In [23]:

# Include the SUI name here for the purpose of merging in the existing SUIs, we can remove them before saving
CODE_SUIs_array = np.concatenate([mp_only[['start_CodeID','start_SUI','start_CUI','start_name']].values,
                                  mp_only[['end_CodeID','end_SUI','end_CUI','end_name']].values])

CODE_SUIs = pd.DataFrame(CODE_SUIs_array,columns=[':START_ID',':END_ID','CUI','name'])

CODE_SUIs[':TYPE']  = 'Term'

CODE_SUIs.drop_duplicates(inplace=True)


# Split CODE_SUIs into 2 df's, one with Terms that were already in UMLS and one that doesnt.
CODE_SUIs_overlap = CODE_SUIs[CODE_SUIs['name'].isin(UMLS_SUIs['name'])]

CODE_SUIs_ok = CODE_SUIs[~CODE_SUIs['name'].isin(UMLS_SUIs['name'])]

# Rename column so we can merge on it
#term_overlap.rename(columns={'Term':'name'},inplace=True)


# Drop ':END_ID' column (aka the old SUIs) 
# and MERGE in the correct SUIs
CODE_SUIs_overlap_fixed = pd.merge(CODE_SUIs_overlap.drop(':END_ID',axis=1),UMLS_SUIs[['SUI:ID','name']],on='name')


# These SUIs will now be the new ':END_ID' column
CODE_SUIs_overlap_fixed.rename(columns={'SUI:ID':':END_ID'},inplace=True)


# Drop 'name' from both df's now  
CODE_SUIs_overlap_fixed.drop('name',axis=1,inplace=True)
CODE_SUIs_ok.drop('name',axis=1,inplace=True)

# Concatenate them back together.
CODE_SUIs_final = pd.concat([CODE_SUIs_ok,CODE_SUIs_overlap_fixed])

# Save
CODE_SUIs_final.to_pickle(output_dir+'MPO/CODE_SUIs_mp_ont.pickle')